In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data   #引用自定义引入数据的方法

In [10]:
#读取数据，存储到mnist中[0 0 0 0 1 0 0 0 0 0]=>4  词向量
mnist      = input_data.read_data_sets('data/', one_hot=True) 
print(mnist)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E1A05F1F60>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E1A05F1A58>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E1A05F1F98>)


In [11]:
#下载下来的数据集被分三个子集：
#5.5W行的训练数据集（mnist.train），
#5千行的验证数据集（mnist.validation)
#1W行的测试数据集（mnist.test）。
#因为每张图片为28x28的黑白图片，所以每行为784维的向量。
trainimg   = mnist.train.images                               
trainlabel = mnist.train.labels
testimg    = mnist.test.images
testlabel  = mnist.test.labels
print ("MNIST loaded")

MNIST loaded


In [12]:
print (trainimg.shape)
print (trainlabel.shape)
print (testimg.shape)
print (testlabel.shape)
print(mnist.train.next_batch)
#print (trainimg)
print (trainlabel[0])

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)
<bound method DataSet.next_batch of <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001E1A05F1F60>>
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [19]:
x=tf.placeholder("float",[None,784],name='x')#784是维度，none表示的是无限多
y=tf.placeholder("float",[None,10],name='y')

W=tf.Variable(tf.zeros([784,10]),name='W')#每个数字是784像素点的，所以w与x相乘的话也要有784个，b-10表示这个10分类的
b=tf.Variable(tf.zeros([10]),name='b')
#回归模型  w*x+b
#tf.matmul:矩阵积
#http://wiki.jikexueyuan.com/project/tensorflow-zh/tutorials/mnist_beginners.html
actv=tf.nn.softmax(tf.matmul(x,W)+b)
#cost function 均值
#reduce_sum 矩阵和，reduction_indices是指沿tensor的哪些维度求和。
#cost 损失函数
cost=tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv),reduction_indices=1))
#优化
learning_rate=0.01
#使用梯度下降，最小化误差
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [20]:
#tf.argmax:对矩阵按行或列计算最大值
#tf.equal:是对比这两个矩阵或者向量的相等的元素，如果是相等的那就返回True，反正返回False，返回的值的矩阵维度和A是一样的
pred=tf.equal(tf.argmax(actv,1),tf.argmax(y,1))
#正确率
accr=tf.reduce_mean(tf.cast(pred,"float"))
#初始化
init=tf.global_variables_initializer()

In [21]:
# 每多少次迭代显示一次损失
training_epochs=50
#批尺寸
batch_size=100
# 训练迭代次数
display_step=5
#session
sess=tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost=0.
    #55000/100
    num_batch=int(mnist.train.num_examples/batch_size)
    for i in range(num_batch):
        #获取数据集 next_batch获取下一批的数据
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        #模型训练
        sess.run(optm,feed_dict={x:batch_xs,y:batch_ys})
        feeds={x:batch_xs,y:batch_ys}
        avg_cost+=sess.run(cost,feed_dict=feeds)/num_batch
    #满足5次的一个迭代
    if epoch % display_step == 0:
        feeds_train = {x: batch_xs, y: batch_ys}
        feeds_test = {x: mnist.test.images, y: mnist.test.labels}
        train_acc = sess.run(accr, feed_dict=feeds_train)
        test_acc = sess.run(accr, feed_dict=feeds_test)
        print ("Epoch: %03d/%03d cost: %.9f train_acc: %.3f test_acc: %.3f" 
               % (epoch, training_epochs, avg_cost, train_acc, test_acc))
writer = tf.summary.FileWriter("D://logs",tf.get_default_graph())
writer.close()
print ("DONE")



Epoch: 000/050 cost: 1.176636985 train_acc: 0.800 test_acc: 0.853
Epoch: 005/050 cost: 0.440966198 train_acc: 0.900 test_acc: 0.895
Epoch: 010/050 cost: 0.383321417 train_acc: 0.910 test_acc: 0.904
Epoch: 015/050 cost: 0.357293261 train_acc: 0.870 test_acc: 0.909
Epoch: 020/050 cost: 0.341462362 train_acc: 0.890 test_acc: 0.912
Epoch: 025/050 cost: 0.330516948 train_acc: 0.940 test_acc: 0.914
Epoch: 030/050 cost: 0.322354329 train_acc: 0.940 test_acc: 0.916
Epoch: 035/050 cost: 0.315942590 train_acc: 0.940 test_acc: 0.917
Epoch: 040/050 cost: 0.310719714 train_acc: 0.960 test_acc: 0.917
Epoch: 045/050 cost: 0.306362533 train_acc: 0.950 test_acc: 0.918
DONE
